### Importing the necessary Python Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error

# !unzip "gro_homework.zip"

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Data preprocessing using Pandas

#### Loading the 'csv' files into pandas dataframes

In [ ]:
df_precipitation = pd.read_csv("gro_homework/Daily Precipitation.csv")
df_soilmoisture = pd.read_csv("gro_homework/Daily Soil Mositure.csv")
df_temperature = pd.read_csv("gro_homework/Daily Temperature.csv")
df_EightdayNDVI = pd.read_csv("gro_homework/Eight Day NDVI.csv")
df_production = pd.read_csv("gro_homework/Production Quantity.csv")

### Creating a new Dataframe of processed data

### Visualizing dataframes

In [ ]:
print(df_precipitation.head(5))
print(df_soilmoisture.head(5))
print(df_temperature.head(5))
print(df_EightdayNDVI.head(5))
print(df_production.head(5))

                 start_date                  end_date     precip  region_id
0  2014-01-01T00:00:00.000Z  2014-01-01T00:00:00.000Z   1.392393         93
1  2014-01-02T00:00:00.000Z  2014-01-02T00:00:00.000Z   0.315380         93
2  2014-01-03T00:00:00.000Z  2014-01-03T00:00:00.000Z   2.347846         93
3  2014-01-04T00:00:00.000Z  2014-01-04T00:00:00.000Z  21.466357         93
4  2014-01-05T00:00:00.000Z  2014-01-05T00:00:00.000Z  32.823651         93
                 start_date                  end_date      smos  region_id
0  2014-01-01T00:00:00.000Z  2014-01-01T00:00:00.000Z  0.310787         93
1  2014-01-02T00:00:00.000Z  2014-01-02T00:00:00.000Z  0.192271         93
2  2014-01-03T00:00:00.000Z  2014-01-03T00:00:00.000Z  0.265683         93
3  2014-01-04T00:00:00.000Z  2014-01-04T00:00:00.000Z  0.265683         93
4  2014-01-05T00:00:00.000Z  2014-01-05T00:00:00.000Z  0.230782         93
                 start_date                  end_date       temp  region_id
0  2014-01-02T00:0

In [ ]:
# Joining the dataframes and resetting the indices
new_df = df_precipitation.merge(df_soilmoisture, how="left", left_on=["start_date","end_date", "region_id"], right_on=["start_date","end_date", "region_id"])
new_df = new_df.reset_index(drop=True)

In [ ]:
new_df = new_df.merge(df_temperature, how="left", left_on=["start_date","end_date", "region_id"], right_on=["start_date","end_date", "region_id"])
new_df = new_df.reset_index(drop=True)

In [ ]:
new_df = new_df.merge(df_EightdayNDVI, how="left", left_on=["start_date", "region_id"], right_on=["start_date", "region_id"])
new_df = new_df.reset_index(drop=True)

In [ ]:
new_df.drop(columns=["end_date_y"], inplace=True)
new_df.rename(columns={"end_date_x":"end_date"}, inplace=True)

### Assumption

Assumption1: the soil moisture is dependent on the precipitation value. Meaning, if today it rains, then the soil will be moist and will still be slightly moist for the coming few days. With this I did a 'forward fill'. 

assumption2: For the temperature, I have considered the mean value over all the days

assumption3: For the NDVI value, I have extrapolated the value for all the remaining 7 days for every data entry. 

In [ ]:
new_df['smos'] = new_df['smos'].fillna(method='ffill', axis=0).fillna(method='bfill',axis=0)

In [ ]:
new_df = new_df.fillna({'temp':new_df['temp'].mean()})

In [ ]:
new_df['ndvi'] = new_df['ndvi'].fillna(method='ffill', axis=0)

In [ ]:
new_df.columns

Index(['start_date', 'end_date', 'precip', 'region_id', 'smos', 'temp',
       'ndvi'],
      dtype='object')

In [ ]:
new_df = new_df.merge(df_production, how="left", left_on=["start_date", "region_id"], right_on=["start_date", "region_id"])

In [ ]:
df_production.head()

,start_date,end_date,prod,region_id
0,2015-01-01T00:00:00.000Z,2015-01-31T00:00:00.000Z,171725,93
1,2015-02-01T00:00:00.000Z,2015-02-28T00:00:00.000Z,188325,93
2,2015-03-01T00:00:00.000Z,2015-03-31T00:00:00.000Z,247856,93
3,2015-04-01T00:00:00.000Z,2015-04-30T00:00:00.000Z,282791,93
4,2015-05-01T00:00:00.000Z,2015-05-31T00:00:00.000Z,291057,93


In [ ]:
# To check if the data processing was doen appropriately
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29940 entries, 0 to 29939
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   start_date  29940 non-null  object 
 1   end_date    29940 non-null  object 
 2   precip      29940 non-null  float64
 3   region_id   29940 non-null  int64  
 4   smos        29940 non-null  float64
 5   temp        29940 non-null  float64
 6   ndvi        29940 non-null  float64
 7   prod        720 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 2.1+ MB


In [ ]:
new_df.drop(columns=["end_date_y"], inplace=True)
new_df.rename(columns={"end_date_x":"end_date"}, inplace=True)

In [ ]:
new_df_2 = new_df[new_df['start_date'] >= min(df_production['start_date'])]

In [ ]:
new_df_2.reset_index(drop=True,inplace=True)

In [ ]:
new_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26290 entries, 0 to 26289
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   start_date  26290 non-null  datetime64[ns, UTC]
 1   end_date    26290 non-null  object             
 2   precip      26290 non-null  float64            
 3   region_id   26290 non-null  int64              
 4   smos        26290 non-null  float64            
 5   temp        26290 non-null  float64            
 6   ndvi        26290 non-null  float64            
 7   prod        720 non-null    float64            
dtypes: datetime64[ns, UTC](1), float64(5), int64(1), object(1)
memory usage: 1.6+ MB


Splitting into the train and the test sets. 

assumption: I have assumed that the training was done in the time periods - '2015-2020' for which the production capacity values were readily available.

assumption; For the test set, I have assumed that the prediction was done only for 1 year from 2021-01-01 to 2021-12-31.

In [ ]:
new_df_2_train = new_df_2[new_df_2['start_date'] < "2021-01-01T00:00:00.000Z"]
new_df_2_test = new_df_2[new_df_2['start_date'] >= "2021-01-01T00:00:00.000Z"] 
new_df_2_test = new_df_2_test[new_df_2_test['start_date'] < "2022-01-01T00:00:00.000Z"] 

assumption: for the production capacity values, I have forward filled the values for all the days in the month.

In [ ]:
new_df_2_train['prod'] = new_df_2_train['prod'].fillna(method='ffill', axis=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
new_df_2.head(5)

,start_date,end_date,precip,region_id,smos,temp,ndvi,prod
0,2015-01-01 00:00:00+00:00,2015-01-01T00:00:00.000Z,2.278169,93,0.397555,26.542191,0.734261,171725.0
1,2015-01-02 00:00:00+00:00,2015-01-02T00:00:00.000Z,0.863206,93,0.328829,22.460309,0.734261,NaN
2,2015-01-03 00:00:00+00:00,2015-01-03T00:00:00.000Z,0.312079,93,0.258153,26.542191,0.734261,NaN
3,2015-01-04 00:00:00+00:00,2015-01-04T00:00:00.000Z,0.411836,93,0.258153,26.542191,0.734261,NaN
4,2015-01-05 00:00:00+00:00,2015-01-05T00:00:00.000Z,2.275593,93,0.200344,26.542191,0.734261,NaN


### Dropping the date related columns .... Not considering them in regression testing

In [ ]:
new_df_2_train.drop(columns=['start_date','end_date'],inplace=True)
new_df_2_test.drop(columns=['start_date','end_date'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Standardizing the four independent variables

In [ ]:
train_mean = new_df_2_train[['precip','smos','temp','ndvi','prod']].mean(axis=0)
train_std = new_df_2_train[['precip','smos','temp','ndvi','prod']].std(axis=0)

In [ ]:
new_df_2_train[['precip','smos','temp','ndvi','prod']] = (new_df_2_train[['precip','smos','temp','ndvi','prod']] - train_mean)/train_std

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
new_df_2_test[['precip','smos','temp','ndvi','prod']] = (new_df_2_test[['precip','smos','temp','ndvi','prod']] - train_mean)/train_std

#### The complete train and test datasets are prepared

In [ ]:
new_df_2_train.head(10)

,precip,region_id,smos,temp,ndvi,prod
0,-0.554952,93,1.478688,-0.034403,-2.04914,0.087733
1,-0.717256,93,0.738559,-2.568207,-2.04914,0.087733
2,-0.780473,93,-0.022555,-0.034403,-2.04914,0.087733
3,-0.769030,93,-0.022555,-0.034403,-2.04914,0.087733
4,-0.555248,93,-0.645119,-0.034403,-2.04914,0.087733
5,0.134715,93,-0.101775,-0.034403,-2.04914,0.087733
6,1.485504,93,-0.101775,-0.416650,-2.04914,0.087733
7,5.434851,93,2.748781,-0.034403,-2.04914,0.087733
8,3.254285,93,2.748781,-0.034403,-1.77281,0.087733
9,-0.602985,93,1.194548,-2.331838,-1.77281,0.087733


In [ ]:
new_df_2_test.tail(10)

,precip,region_id,smos,temp,ndvi,prod
26208,-0.560425,105,2.667813,-0.034403,0.206689,NaN
26209,-0.432496,105,2.667813,-0.214240,0.206689,NaN
26210,-0.697664,105,2.667813,-0.034403,0.206689,NaN
26211,-0.776909,105,1.876430,-0.698539,0.206689,NaN
26212,-0.816227,105,1.876430,-0.034403,0.206689,NaN
26213,-0.816234,105,1.876430,-1.923254,0.206689,NaN
26214,-0.812027,105,1.571025,-2.716382,0.206689,NaN
26215,-0.050879,105,1.571025,-0.034403,0.206689,NaN
26216,7.020468,105,2.416081,-0.034403,0.206689,NaN
26217,-0.000518,105,2.552749,-0.034403,0.206689,NaN


### Train test split starts here

#### Here we create a 20% validation split on the train set

In [ ]:
x_train_2 = new_df_2_train.drop(columns=["prod"]).values
y_train_2 = new_df_2_train["prod"].values
x_test_2 = new_df_2_test.drop(columns=["prod"]).values
y_test_2 = new_df_2_test["prod"].values
X_train_2, X_val_2, Y_train_2, Y_val_2 = train_test_split(x_train_2, y_train_2, test_size=0.2, shuffle=True)

# Data Analysis

### Identifying that Precip does not have a linear relationship with the dependent variable

In brief, Ordinary Least Squares (OLS) Regression compares the difference between individual points in your data set and the predicted best fit line to measure the amount of error produced.

As seen from the summary report below, R-squared is one of the most important measurement produced by this summary. R-squared is the measurement of how much of the independent variable is explained by any changes in our dependent variables. Adjusted R-squared is important for analyzing multiple dependent variables’ efficacy on the model. Since, in linear regression the model’s R-squared value will never go down with additional variables, only equal or higher. Therefore, the model could look more accurate with multiple variables even if they are poorly contributing. The adjusted R-squared penalizes the R-squared formula based on the number of variables, therefore a lower adjusted score may be telling us that some variables are not contributing to our model’s R-squared value.

As seen here, the R-squared value is 0.186, which is very low.

Also looking at the p-value, that is P>|t| tells us that for the feature 'precip' and 'region_id', there is 32.7% and 66.1% chance that the two features do not contribute sufficiently to the dependent variable 'prod'. Hence, we can neglect these two features. 

In [ ]:
regressor_OLS = sm.OLS(endog = Y_train_2,exog = X_train_2).fit()
print(regressor_OLS.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.187
Model:                            OLS   Adj. R-squared (uncentered):              0.187
Method:                 Least Squares   F-statistic:                              808.1
Date:                Wed, 06 Apr 2022   Prob (F-statistic):                        0.00
Time:                        03:03:09   Log-Likelihood:                         -23084.
No. Observations:               17536   AIC:                                  4.618e+04
Df Residuals:                   17531   BIC:                                  4.622e+04
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### Simple Linear without precip column

In [ ]:
x_train_2 = new_df_2_train.drop(columns=['precip','region_id',"prod"]).values
y_train_2 = new_df_2_train["prod"].values
x_test_2 = new_df_2_test.drop(columns=['precip','region_id',"prod"]).values
y_test_2 = new_df_2_test["prod"].values
X_train_2, X_val_2, Y_train_2, Y_val_2 = train_test_split(x_train_2, y_train_2, test_size=0.2, shuffle=True)

In [ ]:
regressor_OLS = sm.OLS(endog = Y_train_2,exog = X_train_2).fit()
print(regressor_OLS.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.187
Model:                            OLS   Adj. R-squared (uncentered):              0.187
Method:                 Least Squares   F-statistic:                              1342.
Date:                Wed, 06 Apr 2022   Prob (F-statistic):                        0.00
Time:                        03:03:17   Log-Likelihood:                         -23091.
No. Observations:               17536   AIC:                                  4.619e+04
Df Residuals:                   17533   BIC:                                  4.621e+04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

From the above summary table, we can see that the R-squared value has slightly improved after dropping, the precipitation values, suggesting that our above hypthesis is true.

No, we move forward with calculating the R-2 squares values and Least Mean Square Errors using different models with three features - "Temperatures", "NDVI", "Soil Moisture".

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train_2,Y_train_2)

LinearRegression()

In [ ]:
def MSE(y,y_pred):
    return ((y-y_pred)**2).mean()

In [ ]:
# Prediction on Training set
X_train_pred_2 = lin_reg.predict(X_train_2)

In [ ]:
# Prediction on Validation sey
X_val_pred_2 = lin_reg.predict(X_val_2)

In [ ]:
print(f'MAPE on train set:{mean_absolute_percentage_error(Y_train_2, X_train_pred_2):.3f}%')

MAPE on train set:1.406%


In [ ]:
print(f'mean square error on validation set:{MSE(Y_val_2,X_val_pred_2):.3f}')

mean square error on validation set:0.799


In [ ]:
print(f'mean square error on training set:{MSE(Y_train_2,X_train_pred_2):.3f}')

mean square error on training set:0.815


In [ ]:
r2_score_LR = lin_reg.score(X_train_2, Y_train_2)
print(f'R2-score of the Linear Regression model: {r2_score_LR:.3f}')

R2-score of the Linear Regression model: 0.187


As seen the error is still high, with the R2-score being low and here we can explore more models. 

### Support Vector Regression

In [ ]:
svr_reg = SVR().fit(X_train_2, Y_train_2)

In [ ]:
X_train_pred_2 = svr_reg.predict(X_train_2)
X_val_pred_2 = svr_reg.predict(X_val_2)

In [ ]:
print(f'MAPE on train set:{mean_absolute_percentage_error(Y_train_2, X_train_pred_2):.3f}%')

MAPE on train set:2.221%


In [ ]:
print(f'mean square error on validation set:{MSE(Y_val_2,X_val_pred_2):.3f}')

mean square error on validation set:0.741


In [ ]:
print(f'mean square error on training set:{MSE(Y_train_2,X_train_pred_2):.3f}')

mean square error on training set:0.769


Here, with the Support Vector Regressor model, the Mean Square Error is reduced on both the training and the validation set. This suggests that this model has trained the data better than the linear Regression. Let us now look at the R2-score, below, there is no improvement with the training even though the mean square error is reduced slightly. Therefore, now look at a different model. 

In [ ]:
r2_score_SVR = svr_reg.score(X_train_2, Y_train_2)
print(f'R2-score of the Support Vector Regression model: {r2_score_LR:.3f}')

R2-score of the Support Vector Regression model: 0.187


### Try polynomial Regression

#### Create Polynomial Features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_f = PolynomialFeatures(degree= 2)
X_train_2_poly = poly_f.fit_transform(X_train_2)
X_val_2_poly = poly_f.fit_transform(X_val_2)
X_test_2_poly = poly_f.fit_transform(x_test_2)

In [ ]:
regressor_OLS = sm.OLS(endog = Y_train_2,exog = X_train_2_poly).fit()
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     698.1
Date:                Wed, 06 Apr 2022   Prob (F-statistic):               0.00
Time:                        03:05:10   Log-Likelihood:                -22217.
No. Observations:               17536   AIC:                         4.445e+04
Df Residuals:                   17526   BIC:                         4.453e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1076      0.009    -11.963      0.0

As seen from the OLS() summary report, the R-squared and the adjusted R-squared error are both improved that the previous models. This time the r-squared score was improved to 0.264. Now we run this model again on our transformed training and validation sets.

In [ ]:
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_train_2_poly,Y_train_2)

LinearRegression()

In [ ]:
X_train_pred_2 = lin_reg_2.predict(X_train_2_poly)
X_val_pred_2 = lin_reg_2.predict(X_val_2_poly)

In [ ]:
MSE(Y_val_2,X_val_pred_2)
print(f'mean square error on validation set:{MSE(Y_val_2,X_val_pred_2):.3f}')

mean square error on validation set:0.715


In [ ]:
print(f'MAPE on train set:{mean_absolute_percentage_error(Y_train_2, X_train_pred_2):.3f}%')

MAPE on train set:1.616%


In [ ]:
print(f'mean square error on training set:{MSE(Y_train_2,X_train_pred_2):.3f}')

mean square error on training set:0.738


As seen the mean square error is further reduced. Now we calculate the R2-square of the polynomial regressor model that we have built above. We see that thePolynomial regression model did better than the SVR or the Linear regressor.

In [ ]:
r2_score_poly = lin_reg_2.score(X_train_2_poly, Y_train_2)
print(f'R2-score of the Polynomial Regression model: {r2_score_poly:.3f}')

R2-score of the Polynomial Regression model: 0.264


Let us look at some more models - 

### EXTRA TREES REGRESSION

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
reg_ETR = ExtraTreesRegressor(n_estimators=2)
reg_ETR.fit(X_train_2,Y_train_2)

ExtraTreesRegressor(n_estimators=2)

In [ ]:
X_train_pred_2 = reg_ETR.predict(X_train_2)
X_val_pred_2 = reg_ETR.predict(X_val_2)

In [ ]:
MSE(Y_val_2,X_val_pred_2)
print(f'mean square error on validation set:{MSE(Y_val_2,X_val_pred_2):.3f}')

mean square error on validation set:0.548


In [ ]:
print(f'mean square error on training set:{MSE(Y_train_2,X_train_pred_2)}')

mean square error on training set:2.0395061787108453e-05


In [ ]:
r2_score_ETR = reg_ETR.score(X_train_2, Y_train_2)
print(f'R2-score of the Polynomial Regression model: {r2_score_ETR}')

R2-score of the Polynomial Regression model: 0.9999796533954146


As seen above the Extra trees Regression model gave the highest R2 score possible with the MSE on validation error as low as ~ $10^{-5}$

This model definitely overfits the training data.
Let us look at a couple more models


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg_RFR = RandomForestRegressor(n_estimators=10)
reg_RFR.fit(X_train_2,Y_train_2)

RandomForestRegressor(n_estimators=10)

In [ ]:
X_train_pred_2 = reg_RFR.predict(X_train_2)
X_val_pred_2 = reg_RFR.predict(X_val_2)

We also calculate the MAPE value for this model.  It the lowest compared to all the above models.

In [ ]:
print(f'MAPE on train set:{mean_absolute_percentage_error(Y_train_2, X_train_pred_2):.3f}%')

MAPE on train set:0.516%


In [ ]:
MSE(Y_val_2,X_val_pred_2)
print(f'mean square error on validation set:{MSE(Y_val_2,X_val_pred_2):.3f}')

mean square error on validation set:0.297


In [ ]:
print(f'mean square error on training set:{MSE(Y_train_2,X_train_pred_2)}')

mean square error on training set:0.058107275809263594


As seen above the training set MSE was low and this model does not seem to overfit on the training data. therefore, we consider this model for prediction on the testing set. Before, that we see the R2-score. As seen below, the score is pretty close to 1, which means the determination of prediction is more accuracte with this model than any other model above.

In [ ]:
r2_score_ETR = reg_ETR.score(X_train_2, Y_train_2)
print(f'R2-score of the Polynomial Regression model: {r2_score_ETR}')

R2-score of the Polynomial Regression model: 0.9999796533954146


### Testing the Model on the test set

In [ ]:
# prediction on Test set
Y_test_pred = reg_RFR.predict(x_test_2)

Now de-normalize the predicted data because the original actual values were normalized before training.

In [ ]:
prod_mean = train_mean[4]
prod_std = train_std[4]

In [ ]:
def denormalize_predictions(y_pred_norm, mean, std):
    return (y_pred_norm*std) + mean

In [ ]:
final_pred = denormalize_predictions(Y_test_pred, prod_mean, prod_std)

we read the ouput csv file into a dataframe

In [ ]:
df_predicted_production = pd.read_csv("gro_homework/predicted_production_qty.csv")

Create a new dataframe with all the dropped columns - "start_date", "end_date", "region_id"

In [ ]:
new_df_test = new_df[new_df['start_date'] >= "2021-01-01T00:00:00.000Z"] 
new_df_test = new_df_test[new_df_test['start_date'] < "2022-01-01T00:00:00.000Z"] 

Here, I have calculated monthly means for all the 10 regions

In [ ]:
regions = new_df_test.loc[:,"region_id"].unique()
new_list = []
for i in regions:
    new_list.append(new_df_test[new_df_test['region_id'] == i].groupby(pd.PeriodIndex(new_df_test.loc[new_df_test['region_id'] ==i]['start_date'], freq='M'))['prod'].mean().values)

Now, I have added this array into the "prod" column of the above output dataframe, which will be written into the csv file.

In [ ]:
np.array(new_list).reshape(-1)
df_predicted_production["prod"] = np.array(new_list).reshape(-1)

In [ ]:
df_predicted_production.to_csv("gro_homework/predicted_production_qty.csv")